In [1]:
import os
from boxsdk import OAuth2, Client

# OAuth 2.0 credentials
client_id = 'vc9saqpuvmiimz3agdvvdcrzzqudrrty'
client_secret = 'eQHxqemYksHP5lrsrhTlntTHFZCGMy5N'
access_token = 'MP8fzoFprx0KLnnIMcJLdx6Uv2JbL4aG'

# Create OAuth2 instance
oauth = OAuth2(
    client_id=client_id,
    client_secret=client_secret,
    access_token=access_token
)

# Create a Box client
client = Client(oauth)

# Function to upload a large file using chunked upload
def upload_large_file(file_path, parent_folder_id):
    file_name = os.path.basename(file_path)
    file_size = os.stat(file_path).st_size
    print(file_path)
    print(file_size)
    # Create a new upload session
    upload_session = client.folder(parent_folder_id).create_upload_session(file_name, 620000744)

    #other code 

    # Commit the upload session
    upload_session.commit()

# Function to upload a file
def upload_file(file_path, parent_folder_id):
    file_name = os.path.basename(file_path)
    file_size = os.stat(file_path).st_size

    if file_size > 20 * 1024 * 1024:
        # For files larger than 20MB, use chunked upload
        upload_large_file(file_path, parent_folder_id)
    else:
        # For files smaller than 20MB, use regular upload
        client.folder(parent_folder_id).upload(file_path, file_name)

# Function to upload a folder and its contents recursively
def upload_folder(folder_path, parent_folder_id):
    folder_name = os.path.basename(folder_path)

    # Create a new folder inside the parent folder
    new_folder = client.folder(parent_folder_id).create_subfolder(folder_name)
    new_folder_id = new_folder.id

    # Iterate over the files and subfolders in the folder
    for item_name in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item_name)

        if os.path.isfile(item_path):
            # Upload the file
            upload_file(item_path, new_folder_id)
        elif os.path.isdir(item_path):
            # Recursively upload the subfolder
            upload_folder(item_path, new_folder_id)

# Example usage: Upload each folder and its contents to "Beamline Sim Data" folder
root_folder = 'E://data/'
parent_folder_id = '215071014756'

# Iterate over the subfolders in the root folder
for folder_name in os.listdir(root_folder):
    folder_path = os.path.join(root_folder, folder_name)
    
    if os.path.isdir(folder_path):
        upload_folder(folder_path, parent_folder_id)


"POST https://upload.box.com/api/2.0/files/upload_sessions" 400 121
{'Server': 'nginx', 'Date': 'Fri, 30 Jun 2023 21:35:22 GMT', 'Content-Type': 'application/json; charset=UTF-8', 'Content-Length': '121', 'x-envoy-upstream-service-time': '4', 'X-Box-Original-Ingress-ADC-Host': 'prod-a-traffic-manager-89fd', 'Strict-Transport-Security': 'max-age=31536000', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}
{'code': '---size',
 'message': 'Invalid file_size: "coord.out"',
 'request_id': '9fb8eb20bda09be407f077386c68d06e'}



E://data/epsnx0.05_alfax2.65_betax234.57_epsny0.10_alfay-0.55_betay170.00_epsnz5.00_alfaz0.10_betaz10.00\coord.out
620000744


BoxAPIException: Message: Invalid file_size: "coord.out"
Status: 400
Code: invalid_file_size
Request ID: 9fb8eb20bda09be407f077386c68d06e
Headers: {'Server': 'nginx', 'Date': 'Fri, 30 Jun 2023 21:35:22 GMT', 'Content-Type': 'application/json; charset=UTF-8', 'Content-Length': '121', 'x-envoy-upstream-service-time': '4', 'X-Box-Original-Ingress-ADC-Host': 'prod-a-traffic-manager-89fd', 'Strict-Transport-Security': 'max-age=31536000', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}
URL: https://upload.box.com/api/2.0/files/upload_sessions
Method: POST
Context Info: None

In [18]:
import requests

# API endpoint for retrieving user information
url = 'https://api.box.com/2.0/users/me'
access_token = '4Ec5xTNCmgMadVUb05I4x0z9jUaGzVuG'

# Authorization header with access token
headers = {
    'Authorization': f'Bearer {access_token}'
}

# Send GET request to retrieve user information
response = requests.get(url, headers=headers)
user_info = response.json()
print(user_info)

# Retrieve the maximum file size limit
upload_limit = user_info['space_amount']

# Print the maximum file size limit
print(f"Maximum file upload size limit: {upload_limit} bytes")


{'type': 'user', 'id': '26079762570', 'name': 'Knight, Ian', 'login': 'iknight@anl.gov', 'created_at': '2023-05-31T12:48:47-07:00', 'modified_at': '2023-06-29T12:24:29-07:00', 'language': 'en', 'timezone': 'America/Chicago', 'space_amount': 1610612736000, 'space_used': 1875887, 'max_upload_size': 53687091200, 'status': 'active', 'job_title': 'Lee Teng', 'phone': '', 'address': '', 'avatar_url': 'https://anl.app.box.com/api/avatar/large/26079762570', 'notification_email': []}
Maximum file upload size limit: 1610612736000 bytes


In [5]:
from boxsdk import OAuth2, Client

# OAuth 2.0 credentials
client_id = 'vc9saqpuvmiimz3agdvvdcrzzqudrrty'
client_secret = 'eQHxqemYksHP5lrsrhTlntTHFZCGMy5N'
access_token = '4Ec5xTNCmgMadVUb05I4x0z9jUaGzVuG'

# Create OAuth2 instance
oauth = OAuth2(
    client_id=client_id,
    client_secret=client_secret,
    access_token=access_token
)

# Create a Box client
client = Client(oauth)

# Function to get all folder names and IDs recursively
def get_all_folder_info(parent_folder_id):
    folder_info = []

    # Get the folder items in the parent folder
    items = client.folder(folder_id=parent_folder_id).get_items()

    for item in items:
        if item.type == 'folder':
            # Append the folder name and ID to the list
            folder_info.append({
                'name': item.name,
                'id': item.id
            })

            # Recursively get folder info in the subfolder
            subfolder_info = get_all_folder_info(item.id)
            folder_info.extend(subfolder_info)

    return folder_info

# Example usage: Get all folder names and IDs in the Box account
root_folder_id = '0'

# Get all folder names and IDs recursively starting from the root folder
all_folder_info = get_all_folder_info(root_folder_id)

# Print the folder names and IDs
for folder in all_folder_info:
    print(f"Folder Name: {folder['name']}, Folder ID: {folder['id']}")


Folder Name: Box Tips, Folder ID: 1146422989
Folder Name: Communications Toolkit, Folder ID: 3895184399
Folder Name: Additional Guidelines, Folder ID: 5699484997
Folder Name: Argonne Public Website Guidelines and References, Folder ID: 51272495974
Folder Name: Drupal 8 Content Templates for new site, Folder ID: 52532659267
Folder Name: Sponsor Guidelines, Folder ID: 127711940675
Folder Name: Argonne Social Medial Profile Banners, Folder ID: 161660260969
Folder Name: LinkedIn Profile Banners, Folder ID: 161660671453
Folder Name: Argonne Virtual Meeting Backgrounds, Folder ID: 116253036058
Folder Name: ANL_75th Anniversary_Meeting Backgrounds, Folder ID: 149866079108
Folder Name: ANL_AET Backgrounds, Folder ID: 161213680386
Folder Name: ANL_ALCF Backgrounds, Folder ID: 149868156158
Folder Name: ANL_APS-U Backgrounds, Folder ID: 149867092535
Folder Name: ANL_Argonne in Chicago Backgrounds, Folder ID: 149866629429
Folder Name: ANL_CNM Backgrounds, Folder ID: 149867520963
Folder Name: ANL_E